In [34]:
from datetime import date

import pandas as pd
# get ingested data
from deeptendies.utils import get_numerical_df

today = date.today()
file = '../stonks/bucket=filesys/topic=GME/version=raw/processed_at=2021-04-04/GME_180.csv'
df = pd.read_csv(file, header=0, index_col=0)
print(df.columns)
dataset = get_numerical_df(df)[-60:]

val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

feature_cols = ['o', 'h', 'l', 'v', 'c']

data_x = data_train[feature_cols].astype('float64')

data_x_val = validation_data[feature_cols].astype(
    'float64')

# Data with train data and the unseen data from subsequent time steps.
data_x_test = dataset[feature_cols].astype('float64')

target_col = 'c'

data_y = data_train[target_col].astype('float64')

data_y_val = validation_data[target_col].astype('float64')

print(data_x.shape)
print(data_y.shape)

"""
The second step is to run the [TimeSeriesForecaster](/time_series_forecaster).
As a quick demo, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.
"""
import autokeras as ak

predict_from = 1
predict_until = 5
lookback = 5
clf = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_from=predict_from,
    predict_until=predict_until,
    max_trials=10,
    objective='val_loss'
)
# Train the TimeSeriesForecaster with train data
clf.fit(x=data_x, y=data_y, validation_data=(data_x_val, data_y_val), batch_size=32,
        epochs=25)
# Predict with the best model(includes original training data).
predictions = clf.predict(data_x_test)
print(predictions.shape)
# Evaluate the best model with testing data.
print(clf.evaluate(data_x_val, data_y_val))

Index(['c', 'h', 'l', 'o', 's', 't', 'v', 'wma'], dtype='object')
(42, 5)
(42,)
INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/25
2/2 [==============================] - 6s 1s/step - loss: 13182.2878 - mean_squared_error: 13182.2878 - val_loss: 703.7216 - val_mean_squared_error: 703.7216
Epoch 2/25
2/2 [==============================] - 0s 66ms/step - loss: 12840.8092 - mean_squared_error: 12840.8092 - val_loss: 1380.4519 - val_mean_squared_error: 1380.4519
Epoch 3/25
2/2 [==============================] - 0s 73ms/step - loss: 11095.7633 - mean_squared_error: 11095.7633 - val_loss: 2142.4099 - val_mean_squared_error: 2142.4099
Epoch 4/25
2/2 [==============================] - 0s 69ms/step - loss: 9530.3340 - mean_squared_error: 9530.3340 - val_loss: 517.0226 - val_mean_squared_error: 517.0226
Epoch 5/25
2/2 [==============================] - 0s 76ms/step - loss: 17090.1686 - mean_squared_error: 17090.1686 - val_loss: 576.0873 - val_mean_squared_error: 576.0873
Epoch 6/25
2/2 [==============================] - 0s 78ms/step - loss: 11626.4421 - mean_squared_error: 11626.4421 - val_loss: 3993.2065 - val_me

INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


(5, 1)
1/1 [==============================] - 1s 1s/step - loss: 502.3798 - mean_squared_error: 502.3798
[502.3797912597656, 502.3797912597656]


In [30]:
!rm -r time_series_forecaster

In [40]:
print(predictions)
print(data_y_val)
data_x_test

# dataset

[[190.06612]
 [190.06612]
 [190.06612]
 [190.06612]
 [190.06612]]
106    246.90
107    265.00
108    260.00
109    264.50
110    220.14
111    208.17
112    209.81
113    201.75
114    200.27
115    194.49
116    181.75
117    120.34
118    183.75
119    181.00
120    181.30
121    194.46
122    189.82
123    191.45
Name: c, dtype: float64


,o,h,l,v,c
64,17.3400,18.9800,17.3300,6056248.0,18.36
65,18.4700,19.4500,18.0200,6129276.0,18.08
66,18.1800,18.3000,17.0800,6481960.0,17.69
67,19.4100,20.6500,19.0100,14927612.0,19.94
68,19.9600,20.4000,19.3168,7060665.0,19.95
69,20.4200,38.6500,20.0300,144501736.0,31.40
70,38.0900,43.0600,33.0500,93717410.0,39.91
71,38.4900,40.7500,34.0100,46866358.0,35.50
72,41.5500,45.5200,36.6400,74721924.0,39.36
73,37.3700,41.1899,36.0600,33471789.0,39.12


In [39]:
import os, pathlib
def df_to_filesys_operator(df, path, fname):
    if not os.path.exists(path):
        pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    df.to_csv(os.path.join(path, fname))
    return

In [ ]:



# staging PRED data tags
ticker, days = 'GME', '180'
bucket = 'bucket=filesys'
topic = 'topic=' + ticker
processed_at = 'processed_at=' + today.strftime("%Y-%m-%d")
version = 'pred'
version = 'version=' + version
filename = ticker + "_" + str(days) + '.csv'

# raw
path = os.path.join(
    bucket,
    topic,
    version,
    processed_at
)

df_to_filesys_operator(df,
                       path,
                       filename)